In [32]:
import numpy as np
from gtoc13 import YPTU, SPTU, KMPDU, bodies_data
from gtoc13.path_finding.binlp.b_utils import min_dv_lam, plot_porkchop
from lamberthub import izzo2015, gooding1990, vallado2013, arora2013, avanzini2008, battin1984
from itertools import product
from IPython.display import clear_output
import time
from pathlib import Path

In [33]:
# Timeframe
Yo = 3
Yf = 15
gridsize = 45
dv_lim = 30  # km/s
generate_all = True
show_plots = False
clear_plots = False
max_count = 5
K = 10
M = 9

In [34]:
To = float(Yo / YPTU)
Tf = float(Yf / YPTU)
ts = np.logspace(To, Tf, gridsize, base=np.e)
dts = np.linspace((0 + (1 / 52)) / YPTU, (Yf - Yo) / YPTU, gridsize)
t_dep = np.log(ts)
tof = dts

In [35]:
keys = [k for k, b in bodies_data.items() if b.is_planet() or b.name == "Yandi"]
KM_idxs = [(k, m) for (k, m) in product(keys, keys) if k != m]

In [37]:
dv_pk = min_dv_lam(K, t_dep[0], M, tof[-1])
dv_izzo = izzo2015(
    1,
    np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    tof[-1],
)
# dv_val = vallado2013(
#     1,
#     np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
#     np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
#     tof[-1],
# )
dv_aror = arora2013(
    1,
    np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    tof[-1],
)
dv_avan = avanzini2008(
    1,
    np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    tof[-1],
)
dv_good = gooding1990(
    1,
    np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    tof[-1],
)
dv_batt = battin1984(
    1,
    np.array(bodies_data[K].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    np.array(bodies_data[M].get_state(t_dep[0], time_units="TU", distance_units="DU").r),
    tof[-1],
)

c:\Users\steph\mambaforge\envs\gtoc\Lib\site-packages\lamberthub\universal_solvers\arora.py:135: RuntimeWarning: invalid value encountered in sqrt
  tof20 = S * np.sqrt(1 - 20 * tau) * (tau + 0.04940968903 * (1 - 20 * tau))
c:\Users\steph\mambaforge\envs\gtoc\Lib\site-packages\lamberthub\universal_solvers\arora.py:136: RuntimeWarning: invalid value encountered in sqrt
  tof100 = S * np.sqrt(1 - 100 * tau) * (tau + 0.00999209404 * (1 - 100 * tau))


In [38]:
print(t_dep[0], tof[-1])
print("pykep", dv_pk)
print("izzo", np.linalg.norm(dv_izzo[0]) + np.linalg.norm(dv_izzo[1]))
# print("vallado", np.linalg.norm(dv_val[0]) + np.linalg.norm(dv_val[1]))
print("arora", np.linalg.norm(dv_aror[0]) + np.linalg.norm(dv_aror[1]))
print("avanzini", np.linalg.norm(dv_avan[0]) + np.linalg.norm(dv_avan[1]))
print("gooding", np.linalg.norm(dv_good[0]) + np.linalg.norm(dv_good[1]))
print("battin", np.linalg.norm(dv_batt[0]) + np.linalg.norm(dv_batt[1]))

19.314682362064598 77.25872944825839
pykep 4.679806315311147
izzo 4.76495662151623
arora 4.764956621645322
avanzini 4.764956621516236
gooding 4.76495662151623
battin 4.764719382425973


In [39]:
0.1 * KMPDU / SPTU

Array(3.05202271, dtype=float64, weak_type=True)

In [ ]:
# if generate_all:
#     count = 0
#     dvs = {}
#     for km in KM_idxs:
#         count += 1
#         k, m = km
#         min_dvs = [[min_dv_lam(k, ti, m, ti + dt) for dt in tof] for ti in t_dep]
#         min_idx = np.unravel_index(np.argmin(min_dvs), (len(t_dep), len(tof)))
#         dvs[km] = (
#             np.round(min_dvs[min_idx[0]][min_idx[1]], 3),
#             (
#                 np.round(t_dep[min_idx[0]] * float(YPTU), 3),
#                 np.round(tof[min_idx[1]] * float(YPTU), 3),
#             ),
#         )
#         plot_porkchop(
#             body1=k, body2=m, t_dep=t_dep, tof=tof, dvs=min_dvs, dv_limit=dv_lim, show=show_plots
#         )
#         if clear_plots and count < 5:
#             time.sleep(5)
#             clear_output(wait=True)
#             if count == max_count:
#                 break
# else:
#     dvs = [[min_dv_lam(K, ti, M, dt) for dt in tof] for ti in t_dep]
#     plot_porkchop(body1=K, body2=M, t_dep=t_dep, tof=tof, dvs=dvs, dv_limit=dv_lim, show=show_plots)
#     print(np.round(np.min(dvs), 3), np.round(np.max(dvs), 3))
#     print(t_dep)

In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: x[1][0])
# with open(Path.cwd() / "outputs/by_min_dv.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")

((6, 5), (0.119, (15.0, 12.0)))
((5, 6), (0.145, (3.0, 12.0)))
((7, 5), (0.155, (15.0, 2.742)))
((7, 6), (0.169, (15.0, 12.0)))
((4, 1000), (0.175, (3.0, 2.198)))
((1000, 4), (0.218, (6.818, 0.292)))
((1000, 6), (0.234, (10.636, 0.564)))
((5, 1000), (0.24, (3.0, 10.094)))
((5, 4), (0.264, (3.0, 4.92)))
((1000, 5), (0.276, (3.0, 6.554)))
((7, 1000), (0.283, (4.909, 11.728)))
((8, 6), (0.3, (15.0, 12.0)))
((4, 6), (0.306, (7.636, 8.46)))
((6, 1000), (0.307, (3.0, 9.822)))
((5, 7), (0.312, (15.0, 1.108)))
((7, 4), (0.312, (4.364, 10.911)))
((4, 5), (0.333, (7.636, 0.019)))
((8, 5), (0.342, (15.0, 12.0)))
((1000, 7), (0.344, (8.727, 12.0)))
((1000, 8), (0.344, (13.364, 12.0)))
((6, 7), (0.35, (15.0, 12.0)))
((5, 8), (0.358, (11.182, 12.0)))
((6, 4), (0.362, (3.0, 4.104)))
((4, 7), (0.374, (12.273, 12.0)))
((6, 8), (0.379, (15.0, 12.0)))
((8, 1000), (0.383, (15.0, 12.0)))
((4, 8), (0.406, (14.455, 12.0)))
((8, 4), (0.419, (12.818, 12.0)))
((5, 3), (0.459, (3.545, 1.108)))
((3, 5), (0.476, (

In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: (x[1][1][0], x[1][0]))
# with open(Path.cwd() / "outputs/by_starting_t_and_dv.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")


((5, 6), (0.145, (3.0, 12.0)))
((4, 1000), (0.175, (3.0, 2.198)))
((5, 1000), (0.24, (3.0, 10.094)))
((5, 4), (0.264, (3.0, 4.92)))
((1000, 5), (0.276, (3.0, 6.554)))
((6, 1000), (0.307, (3.0, 9.822)))
((6, 4), (0.362, (3.0, 4.104)))
((7, 3), (0.562, (3.0, 10.911)))
((1000, 3), (0.605, (3.0, 0.836)))
((2, 5), (0.614, (3.0, 1.108)))
((4, 2), (0.729, (3.0, 0.019)))
((2, 1), (2.7, (3.0, 0.019)))
((7, 2), (0.59, (3.273, 11.728)))
((4, 3), (0.631, (3.273, 0.564)))
((2, 4), (0.774, (3.273, 0.019)))
((2, 3), (0.854, (3.273, 0.836)))
((7, 1), (1.846, (3.273, 9.549)))
((3, 1), (2.526, (3.273, 0.019)))
((5, 3), (0.459, (3.545, 1.108)))
((6, 3), (0.511, (3.545, 4.92)))
((3, 1000), (0.545, (3.545, 0.019)))
((3, 2), (0.866, (3.545, 0.564)))
((1, 4), (2.082, (3.818, 2.742)))
((1, 3), (2.548, (3.818, 0.564)))
((5, 2), (0.593, (4.091, 2.47)))
((6, 2), (0.621, (4.091, 7.916)))
((3, 4), (0.66, (4.091, 0.019)))
((7, 4), (0.312, (4.364, 10.911)))
((3, 5), (0.476, (4.636, 0.292)))
((7, 1000), (0.283, (4.90

In [ ]:
# min_dv_arcs = sorted([(k, v) for k, v in dvs.items()], key=lambda x: (x[1][1][0], x[1][1][1]))
# with open(Path.cwd() / "outputs/by_starting_t_and_dt.txt", "w+") as f:
#     f.write("((ID1, ID2),(min_dv,(t_dep, tof)))")
#     for each in min_dv_arcs:
#         print(each)
#         f.write(str(each) + "\n")

((2, 1), (2.7, (3.0, 0.019)))
((4, 2), (0.729, (3.0, 0.019)))
((1000, 3), (0.605, (3.0, 0.836)))
((2, 5), (0.614, (3.0, 1.108)))
((4, 1000), (0.175, (3.0, 2.198)))
((6, 4), (0.362, (3.0, 4.104)))
((5, 4), (0.264, (3.0, 4.92)))
((1000, 5), (0.276, (3.0, 6.554)))
((6, 1000), (0.307, (3.0, 9.822)))
((5, 1000), (0.24, (3.0, 10.094)))
((7, 3), (0.562, (3.0, 10.911)))
((5, 6), (0.145, (3.0, 12.0)))
((2, 4), (0.774, (3.273, 0.019)))
((3, 1), (2.526, (3.273, 0.019)))
((4, 3), (0.631, (3.273, 0.564)))
((2, 3), (0.854, (3.273, 0.836)))
((7, 1), (1.846, (3.273, 9.549)))
((7, 2), (0.59, (3.273, 11.728)))
((3, 1000), (0.545, (3.545, 0.019)))
((3, 2), (0.866, (3.545, 0.564)))
((5, 3), (0.459, (3.545, 1.108)))
((6, 3), (0.511, (3.545, 4.92)))
((1, 3), (2.548, (3.818, 0.564)))
((1, 4), (2.082, (3.818, 2.742)))
((3, 4), (0.66, (4.091, 0.019)))
((5, 2), (0.593, (4.091, 2.47)))
((6, 2), (0.621, (4.091, 7.916)))
((7, 4), (0.312, (4.364, 10.911)))
((3, 5), (0.476, (4.636, 0.292)))
((1, 5), (1.645, (4.909, 